In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
#from psycopg2 import extras as ex
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from sklearn.preprocessing import *
import datetime

## prepare data

In [2]:
# connect to the postgresql
db_connection = psycopg2.connect(host='127.0.0.1',dbname="chicago_business_intelligence", user="postgres" , password="12345")
cursor = db_connection.cursor()

In [3]:
# get data
select_data = (
    """
        SELECT * FROM "neighborhood_community_zip";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_1 = pd.DataFrame(result_1)
df_1.columns = ["id","geo_id","zipcode","community","neighborhood"]
df_1 = df_1[["zipcode","community","neighborhood"]]
df_1.zipcode = df_1.zipcode.astype("int")
df_1

,zipcode,community,neighborhood
0,60601,Loop,Printer's Row
1,60602,Loop,Printer's Row
2,60603,Loop,Printer's Row
3,60604,Loop,Printer's Row
4,60605,Loop,Printer's Row
...,...,...,...
306,60652,Ashburn,Parkview
307,60652,Ashburn,Scottsdale
308,60652,Ashburn,Wrightwood
309,60656,O'Hare,Schorsch Forest View


In [4]:
# get data
select_data = (
    """
        SELECT * FROM "taxi_trip_total";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_2 = pd.DataFrame(result_1)
df_2.columns = ["id","trip_id","trip_start_timestamp","trip_end_timestamp","pickup_centroid_latitude","pickup_centroid_longitude",
                   "dropoff_centroid_latitude","dropoff_centroid_longitude","pickup_zip_code","dropoff_zip_code"]
df_2 = df_2[["trip_start_timestamp","trip_end_timestamp","pickup_zip_code","dropoff_zip_code"]]
df_2.pickup_zip_code = df_2.pickup_zip_code.astype("int")
df_2.dropoff_zip_code = df_2.dropoff_zip_code.astype("int")
df_2

,trip_start_timestamp,trip_end_timestamp,pickup_zip_code,dropoff_zip_code
0,2022-03-01 00:00:00+08:00,2022-03-01 00:00:00+08:00,60666,60666
1,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60610,60657
2,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60657,60616
3,2022-03-01 00:00:00+08:00,2022-03-01 01:00:00+08:00,60610,60610
4,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60666,60604
...,...,...,...,...
3239,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60607,60603
3240,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60643,60616
3241,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60622,60610
3242,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60610,60604


In [5]:
# get data
select_data = (
    """
        SELECT * FROM "covid_ccvi";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_3 = pd.DataFrame(result_1)
df_3.columns = ["id","community_area_name","ccvi_score","ccvi_category"]
df_3 = df_3[["community_area_name","ccvi_category"]]
df_3.columns = ["community","ccvi_category"]
df_3

,community,ccvi_category
0,Ashburn,MEDIUM
1,Rogers Park,LOW
2,Lake View,LOW
3,Jefferson Park,LOW
4,Archer Heights,HIGH
...,...,...
72,North Center,LOW
73,Calumet Heights,MEDIUM
74,Irving Park,LOW
75,Greater Grand Crossing,MEDIUM


## process data

In [6]:
#divide into pickup and dropoff
df_2_pick = df_2[["trip_start_timestamp","trip_end_timestamp","pickup_zip_code"]]
df_2_pick.columns = ["trip_start_timestamp","trip_end_timestamp","zipcode"]
df_2_off = df_2[["trip_start_timestamp","trip_end_timestamp","dropoff_zip_code"]]
df_2_off.columns = ["trip_start_timestamp","trip_end_timestamp","zipcode"]

In [7]:
# pickup
df_2_pick_1 = df_2_pick.merge(df_1,how='inner', on='zipcode')
df_2_pick_1["type"] = "pick"
# drop
df_2_drop_1 = df_2_off.merge(df_1,how='inner', on='zipcode')
df_2_drop_1["type"] = "drop"
#merge
df_pick_drop = df_2_pick_1.append(df_2_drop_1)
df_pick_drop = df_pick_drop.reset_index(drop=True)
df_pick_drop
#add ccvi
df_pick_drop = df_pick_drop.merge(df_3,how='inner', on='community')
df_pick_drop

,trip_start_timestamp,trip_end_timestamp,zipcode,community,neighborhood,type,ccvi_category
0,2022-03-01 00:00:00+08:00,2022-03-01 00:00:00+08:00,60666,O'Hare,Schorsch Forest View,pick,LOW
1,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60666,O'Hare,Schorsch Forest View,pick,LOW
2,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60666,O'Hare,Schorsch Forest View,pick,LOW
3,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60666,O'Hare,Schorsch Forest View,pick,LOW
4,2022-03-01 00:00:00+08:00,2022-03-01 00:30:00+08:00,60666,O'Hare,Schorsch Forest View,pick,LOW
...,...,...,...,...,...,...,...
40319,2022-02-28 23:45:00+08:00,2022-03-01 00:00:00+08:00,60634,Montclare,Montclare,drop,HIGH
40320,2022-02-28 23:30:00+08:00,2022-02-28 23:45:00+08:00,60634,Montclare,Montclare,drop,HIGH
40321,2022-02-28 23:15:00+08:00,2022-02-28 23:30:00+08:00,60634,Montclare,Montclare,drop,HIGH
40322,2022-02-28 23:15:00+08:00,2022-02-28 23:30:00+08:00,60634,Montclare,Montclare,drop,HIGH


In [8]:
# get hig ccvi_categore
df_pick_drop_high_ccvi = df_pick_drop[df_pick_drop["ccvi_category"]=="HIGH"]
df_pick_drop_high_ccvi = df_pick_drop_high_ccvi.reset_index(drop=True)
df_pick_drop_high_ccvi

,trip_start_timestamp,trip_end_timestamp,zipcode,community,neighborhood,type,ccvi_category
0,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60638,Garfield Ridge,Le Claire Courts,pick,HIGH
1,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60638,Garfield Ridge,Sleepy Hollow,pick,HIGH
2,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60638,Garfield Ridge,Vittum Park,pick,HIGH
3,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60638,Garfield Ridge,Le Claire Courts,pick,HIGH
4,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60638,Garfield Ridge,Sleepy Hollow,pick,HIGH
...,...,...,...,...,...,...,...
3669,2022-02-28 23:45:00+08:00,2022-03-01 00:00:00+08:00,60634,Montclare,Montclare,drop,HIGH
3670,2022-02-28 23:30:00+08:00,2022-02-28 23:45:00+08:00,60634,Montclare,Montclare,drop,HIGH
3671,2022-02-28 23:15:00+08:00,2022-02-28 23:30:00+08:00,60634,Montclare,Montclare,drop,HIGH
3672,2022-02-28 23:15:00+08:00,2022-02-28 23:30:00+08:00,60634,Montclare,Montclare,drop,HIGH


## report

In [9]:
# neighborhoodsccvi_category
rp_df_pick_drop_high_ccvi = df_pick_drop_high_ccvi[['trip_start_timestamp',"neighborhood","type","ccvi_category"
                        ]].groupby(by=["trip_start_timestamp","neighborhood","type"],dropna=False)["ccvi_category"].count().reset_index()
rp_df_pick_drop_high_ccvi.columns=['trip_start_timestamp',"neighborhood","type","ccvi_high_count"]
rp_df_pick_drop_high_ccvi

,trip_start_timestamp,neighborhood,type,ccvi_high_count
0,2022-02-28 19:15:00+08:00,Belmont Gardens,drop,4
1,2022-02-28 19:15:00+08:00,Brainerd,drop,3
2,2022-02-28 19:15:00+08:00,Fernwood,drop,1
3,2022-02-28 19:15:00+08:00,Fifth City,pick,1
4,2022-02-28 19:15:00+08:00,Galewood,drop,1
...,...,...,...,...
590,2022-03-01 00:00:00+08:00,Lawndale,drop,1
591,2022-03-01 00:00:00+08:00,Le Claire Courts,pick,2
592,2022-03-01 00:00:00+08:00,Little Village,drop,1
593,2022-03-01 00:00:00+08:00,Sleepy Hollow,pick,2
